# Ejercicio 1

### *Estandariza, identifica y enumera cada uno de los atributos/variables de la estructura del archivo "Web_acces_log-akumenius.com" que encontraras en el repositorio de GitHub "Data-sources"*

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
# hacemos una primera visualizacion
pd.set_option('display.max_colwidth', None)
data = pd.read_csv('Web_access_log-akumenius.com.txt', names=["registros"])
data.head()

,registros
0,"localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1.0"" 200 - ""-"" ""Apache (internal dummy connection)"" VLOG=-"
1,"localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1.0"" 200 - ""-"" ""Apache (internal dummy connection)"" VLOG=-"
2,"localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1.0"" 200 - ""-"" ""Apache (internal dummy connection)"" VLOG=-"
3,"localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1.0"" 200 - ""-"" ""Apache (internal dummy connection)"" VLOG=-"
4,"localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] ""OPTIONS * HTTP/1.0"" 200 - ""-"" ""Apache (internal dummy connection)"" VLOG=-"


In [3]:
# usamos Regex para separar y definir los atributos y visualizamos los 5 primeros registros
data = pd.read_csv(
    'Web_access_log-akumenius.com.txt',
    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
    engine='python',
    na_values="-",
    header=None,
    usecols=[0, 1, 4, 5, 6, 7, 8, 9],
    names=['host','ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'])

data.head()

,host,ip,time,request,status,size,referer,user_agent
0,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
1,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
2,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
3,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
4,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""


In [4]:
# visualizamos los 5 ultimos registros
data.tail()

,host,ip,time,request,status,size,referer,user_agent
261868,www.akumenius.com,5.255.253.53,[02/Mar/2014:03:05:39 +0100],"""GET / HTTP/1.1""",200,7528.0,"""-""","""Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)"""
261869,www.akumenius.com,74.86.158.107,[02/Mar/2014:03:09:52 +0100],"""HEAD / HTTP/1.1""",200,NaN,"""-""","""Mozilla/5.0+(compatible; UptimeRobot/2.0; http://www.uptimerobot.com/)"""
261870,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
261871,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
261872,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""


# Ejercicio 2

### *Limpia, preprocesa, estructura y transforma (dataframe) los datos del registro de acceso a la web*

In [5]:
# instalacion user_agents
try:
  from user_agents import parse
except ImportError:
  !pip install user_agents

In [8]:
from datetime import datetime
from user_agents import parse
from http.client import responses
import numpy as np

df = data.copy()

# extraemos los metodos de la columna request
df["methods"] = df["request"].str.extract(r'(GET|HEAD|TRACE|OPTIONS|POST|PUT|DELETE|CONNECT|PATCH)')

# extraemos las URL de request
df["URL"] = df["request"].str.extract(r'\s(\/+[^\s]+)')

# extraemos el protocolo de request
df["protocol"] = df["request"].str.extract(r'(HTTP\/1\.[0|1])')

# Eliminamos las comillas de la columna referer
df["referer"] = df["referer"].str.replace("\"","")
df["referer"]= df["referer"].apply(lambda x: np.nan if x == "-" else x)

# Analizamos la columna user agent para identificar el buscador, el dispositivo usado y los atributos del sistema operativo
df["user_agent"] = df["user_agent"].str.replace("\"","")
df["browser_family"] = df["user_agent"].apply(lambda x: parse(x).browser.family )
df["browser_version"] = df["user_agent"].apply(lambda x: parse(x).browser.version_string)
df["os_family"] = df["user_agent"].apply(lambda x: parse(x).os.family)
df["os_version"] = df["user_agent"].apply(lambda x: parse(x).os.version_string)
df["device_family"] = df["user_agent"].apply(lambda x: parse(x).device.family)
df["device_brand"] = df["user_agent"].apply(lambda x: parse(x).device.brand)
df["device_model"] = df["user_agent"].apply(lambda x: parse(x).device.model)

df.head()

,host,ip,time,request,status,size,referer,user_agent,methods,URL,protocol,browser_family,browser_version,os_family,os_version,device_family,device_brand,device_model
0,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,NaN,Apache (internal dummy connection),OPTIONS,NaN,HTTP/1.0,Other,,Other,,Other,None,None
1,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,NaN,Apache (internal dummy connection),OPTIONS,NaN,HTTP/1.0,Other,,Other,,Other,None,None
2,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,NaN,Apache (internal dummy connection),OPTIONS,NaN,HTTP/1.0,Other,,Other,,Other,None,None
3,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,NaN,Apache (internal dummy connection),OPTIONS,NaN,HTTP/1.0,Other,,Other,,Other,None,None
4,localhost,127.0.0.1,[23/Feb/2014:03:10:31 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,NaN,Apache (internal dummy connection),OPTIONS,NaN,HTTP/1.0,Other,,Other,,Other,None,None


In [7]:
df.tail()

,host,ip,time,request,status,size,referer,user_agent,methods,URL,protocol,browser_family,browser_version,os_family,os_version,device_family,device_brand,device_model
261868,www.akumenius.com,5.255.253.53,[02/Mar/2014:03:05:39 +0100],"""GET / HTTP/1.1""",200,7528.0,NaN,Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots),GET,NaN,HTTP/1.1,YandexBot,3.0,Other,,Spider,Spider,Desktop
261869,www.akumenius.com,74.86.158.107,[02/Mar/2014:03:09:52 +0100],"""HEAD / HTTP/1.1""",200,NaN,NaN,Mozilla/5.0+(compatible; UptimeRobot/2.0; http://www.uptimerobot.com/),HEAD,NaN,HTTP/1.1,UptimeRobot,2.0,Other,,Spider,Spider,Desktop
261870,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,NaN,Apache (internal dummy connection),OPTIONS,NaN,HTTP/1.0,Other,,Other,,Other,None,None
261871,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,NaN,Apache (internal dummy connection),OPTIONS,NaN,HTTP/1.0,Other,,Other,,Other,None,None
261872,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,NaN,Apache (internal dummy connection),OPTIONS,NaN,HTTP/1.0,Other,,Other,,Other,None,None


In [9]:
pip install pygeoip

Note: you may need to restart the kernel to use updated packages.
